In [1]:
#Imports
import csv
import cv2
import numpy as np
from sklearn.cross_validation import train_test_split
import sklearn
import random
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda,Cropping2D,Dropout,Activation
from keras.layers.convolutional import Convolution2D 
from keras.layers.pooling import MaxPooling2D
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
#Lines will store data as follows Center,Left,Right,Angle,Throttle,Break,Speed (Access from 0-6)
lines=[]
with open("../Data/driving_log.csv") as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)
#Split into sets
TrainSamples, ValidationSamples=train_test_split(lines,test_size=0.2)
print("DataSet Ready")

DataSet Ready


In [3]:
print(len(TrainSamples))
print(len(ValidationSamples))

4426
1107


In [4]:
#Collect data from files
SteerCorrection=0.25
Multiplier=3
ImageShape=(160,320,3)
#Generators
def BatchGenerator(Data,BatchSize=1):
    NumSamples=10 #len(Data)
    while 1:
        random.shuffle(Data)                             #Shuffle Data so images come in random order
        for offset in range(0,NumSamples,BatchSize):     #Increment offset by BatchSize
            BatchSample=Data[offset:offset+BatchSize]    #Select batch size number of Data samples
        TrainFeatures=np.zeros((len(BatchSample)*Multiplier,160,320,3))
        TrainLabels=np.zeros(len(BatchSample)*Multiplier)
        i=0
        for Sample in BatchSample:
            #Images
            CenterImg=np.asarray(cv2.imread(Sample[0]))
            #Might be a bug in simulator. Adding extra whitespace to address for left and right
            LeftImg=np.asarray(cv2.imread(Sample[1].strip()))  
            RightImg=np.asarray(cv2.imread(Sample[2].strip()))
            #Steering Angles
            CenterAngle=float(Sample[3])
            LeftAngle=(CenterAngle+SteerCorrection)
            RightAngle=(CenterAngle-SteerCorrection)
            #Image and Angle Accumulation
            TrainFeatures[i]=CenterImg
            TrainLabels[i]=CenterAngle
            i=i+1
            TrainFeatures[i]=LeftImg
            TrainLabels[i]=LeftAngle
            i=i+1
            TrainFeatures[i]=RightImg
            TrainLabels[i]=RightAngle
            yield sklearn.utils.shuffle(TrainFeatures,TrainLabels)

In [5]:
#Converting into numpy Arrays
GeneratedTrain=BatchGenerator(TrainSamples,10)
GeneratedVal=BatchGenerator(ValidationSamples,10)

In [6]:
#Network to drive the car
FilShape=[5,5,5,3,3]
NumFils=[24,36,48,64,64]
FCSize=[200,100,10,1]
model=Sequential()
#PreProcessing
model.add(Lambda(lambda x:x/255.0-0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
#CL1
model.add(Convolution2D(NumFils[0],FilShape[0],FilShape[0]))
model.add(Activation('relu'))
#CL2
model.add(Convolution2D(NumFils[1],FilShape[1],FilShape[1]))
model.add(Activation('relu'))
#CL3
model.add(Convolution2D(NumFils[2],FilShape[2],FilShape[2]))
model.add(Activation('relu'))
#CL4
model.add(Convolution2D(NumFils[3],FilShape[3],FilShape[3]))
#CL5
model.add(Convolution2D(NumFils[4],FilShape[4],FilShape[4]))
#Flatten
model.add(Flatten())
#FC1
model.add(Dense(FCSize[0]))
#model.add(Dropout(0.5))
#FC2
model.add(Dense(FCSize[1]))
#model.add(Dropout(0.5))
#FC3
model.add(Dense(FCSize[2]))
#model.add(Dropout(0.5))
#FC4
model.add(Dense(FCSize[3]))


In [ ]:
#Model
model.compile(loss='mse',optimizer='adam')
model.fit_generator(GeneratedTrain,samples_per_epoch=len(TrainSamples),validation_data=GeneratedVal,
                    nb_val_samples=len(ValidationSamples),nb_epoch=7)
print("Saving Model...")
model.save("Model.h5")
print("Done...")

Epoch 1/7
4410/4426 [============================>.] - ETA: 0s - loss: 4025.5053

/home/krishna/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


4440/4426 [==============================] - 55s - loss: 3998.3064 - val_loss: 0.0287
Epoch 2/7
4440/4426 [==============================] - 47s - loss: 0.0234 - val_loss: 0.0311
Epoch 3/7
 360/4426 [=>............................] - ETA: 39s - loss: 0.0200